In [ ]:
import JSON
using Plots
using HDF5
using Trapz
include("../scripts/tools.jl")
include("../scripts/get_grid.jl")
include("../scripts/get_params.jl")

In [ ]:
border = import_border("../data/border.json")
dx = 60
Nx, Ny, xrange, yrange, isinside, isborder, n = get_grid(border, Float64(dx))

x = repeat(reshape(xrange,1,Nx),Ny,1)
y = repeat(reshape(yrange,Ny,1),1,Nx)
x_for_bx = repeat(reshape(xrange,1,Nx),Ny,1) .- dx/2
y_for_bx = repeat(reshape(yrange,Ny,1),1,Nx)
x_for_by = repeat(reshape(xrange,1,Nx),Ny,1)
y_for_by = repeat(reshape(yrange,Ny,1),1,Nx) .+ dx/2;



In [ ]:
recompute_param = true
sigma = 100.
if(recompute_param)
    bx, by, p, m, d = get_params(isinside, isborder, sigma, Float64(dx), "../data/pantagruel.h5",
        "../numerics/grid_params_" * string(dx) * ".h5")
else
    bx, by, p, m, d = get_params(isinside, "../numerics/grid_params_" * string(dx) * ".h5")
end
trapz((yrange, xrange), bx)

In [ ]:
contour(d, fill=true)

In [ ]:
interval = 100
Niter = 5000
th = zeros(Ny, Nx)


@time begin
    for k in 1:Niter
        if(mod(k,interval) == 0)
            temp = copy(th)
        end
        Threads.@threads for i in 2:Ny-1
            Threads.@threads for j in 2:Nx-1
                if(isinside[i,j] | isborder[i,j])
                    bij = (by[i-1,j] + by[i,j] + bx[i,j] + bx[i,j+1])
                    th[i,j] = (by[i,j] * th[i+1,j] + by[i-1,j] * th[i-1,j] + 
                        bx[i,j+1] * th[i,j+1] + bx[i,j] * th[i,j-1] + dx^2*p[i,j]) / bij
                    
                    Threads.@threads for k in 1:size(n,1)
                    #for k in 1:size(n,1)
                        i = Int64(n[k,1])
                        j = Int64(n[k,2])
                        nx = n[k,4]
                        ny = n[k,3]
                        if(nx == 1)
                            th0[i,j] = th0[i,j-2]
                        else(nx == -1)
                            th0[i,j] = th0[i,j+2]
                        end
                        if(ny == 1)
                            th0[i,j] = th0[i-2,j]
                        else(ny == -1)
                            th0[i,j] = th0[i+2,j]
                        end
                    end            
                    
                end
            end
        end   
        if(mod(k,interval) == 0)
            println([k maximum(abs.(th-temp))])
        end

    end
end


In [ ]:
println(maximum(th))
println(minimum(th))
do_plot(isinside, th)
#do_plot(isgrid, p)
#contour(bx, fill=true)

In [ ]:
fid = h5open("../numerics/stable_" * string(dx) * ".h5", "w")
write(fid, "th", th)
close(fid)